In [ ]:
# import geemap
# import ee
# ee.Authenticate()
# ee.Initialize(project='sentinel-treeclassification')

In [7]:
import numpy as np
from pathlib import Path
shards_dir = Path.home().joinpath('/mnt', 'wsl', 'training_data', 'shards')

print(np.load(shards_dir.joinpath('features_B3_B8_2019', 'feature_1.npy'))[0][0])

print(np.load(shards_dir.joinpath('features_B3_B8_2018', 'feature_1.npy'))[0][0])

[0.0876 0.1914]
[0.0857 0.1958]


In [5]:
import geopandas as gpd
gdf = gpd.read_file(Path('labels').joinpath('full_geometries.json'))
# gdf_points = gdf
# gdf_points.geometry = gdf_points.geometry.centroid

In [ ]:
import datetime
import importlib
import ee_downloader
importlib.reload(ee_downloader)

start_date = datetime.datetime(2020, 3, 1)
end_date = datetime.datetime(2024, 3, 1)
ee_downloader.EEDownloader().download_era5(
    gdf, start_date, end_date, Path('weather_data'))

/home/pj/sentinel/ee_downloader.py:130: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_points = gpd.GeoDataFrame(geometry=gdf.geometry.centroid)


In [ ]:
# df = pd.read_csv(Path('weather_data', 'era5_2020.csv'))
# df.interpolate(method='nearest')

In [ ]:
# for eval_year in [2017, 2020, 2021, 2022, 2023]:
    # eval_generator = DataGenerator(
        # utils.selected_classes.index, shuffle=False, year=eval_year, **params)
#     preds_path = model_dir.joinpath(f"preds_{params['year']}.npy")
#     if preds_path.is_file():
#         y_pred = np.load(preds_path)
#     else:
#         y_pred = model.predict(x=eval_generator, verbose=1)
#         np.save(preds_path, y_pred)

#     y_true = utils.selected_classes.iloc[:y_pred.shape[0]].to_numpy()
#     cm = confusion_matrix(y_true.flatten(), (y_pred > 0.5).flatten().astype(int))
#     plot = ConfusionMatrixDisplay(confusion_matrix=cm).plot()
#     display(plot)

In [1]:
import plotly.express as px

class_names = utils.selected_classes.columns
indices = utils.selected_classes.index

preds_path = model_dir.joinpath(f'preds_2020.npy')
y_prev = np.load(preds_path)

weather_prev = pd.read_csv(Path('weather_data', 'era5_2020.csv'))
eval_years = [2021, 2022, 2023]
corrs = []

for eval_year in eval_years:
    preds_path = model_dir.joinpath(f'preds_{eval_year}.npy')
    y_pred = np.load(preds_path)
    y_diff = pd.DataFrame(y_pred - y_prev, columns=class_names)
    
    weather = pd.read_csv(Path('weather_data', f'era5_{eval_year}.csv'))
    weather_diff = ((weather - weather_prev)
                    .loc[indices]
                    .iloc[:y_pred.shape[0]]
                    .iloc[y_diff.index])
    
    corr = y_diff.join(weather_diff).corr(method='pearson').round(2)
    corrs.append(corr)

    y_prev = y_pred
    weather_prev = weather

fig = px.imshow(
    np.array(corrs),
    animation_frame=0,
    labels=dict(color="Corr coef"),
    x=corrs[0].index,
    y=corrs[0].columns,
    title='Annual correlation heatmap',
    text_auto=True, aspect='auto', zmin=0, height=500
)
fig.layout.sliders[0]['currentvalue']['prefix'] = ''
for year, step in zip(eval_years, fig.layout.sliders[0].steps):
    step.label = str(year)

fig

NameError: name 'utils' is not defined

In [ ]:
from glob import glob
from json import load
from pathlib import Path

summation = 0

for f in Path('.').glob('*.ipynb'):
    if 'test' in str(f):
        continue
    cells = load(open(f))["cells"]
    summation += sum(len(c["source"]) for c in cells)
    
for f in Path('.').glob('*.py'):
    summation += sum(1 for _ in open(f))
    
print(summation)

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass